<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_groupchat_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import chromadb

import autogen
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": ["gpt-3.5-turbo", "gpt-35-turbo", "gpt-35-turbo-0613", "gpt-4", "gpt4", "gpt-4-32k"],
    },
)

print("LLM models: ", [config_list[i]["model"] for i in range(len(config_list))])

LLM models:  ['gpt-4-32k', 'gpt-4']


It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well).

The config list looks like the following:
```python
config_list = [
    {
        "model": "gpt-4",
        "api_key": "<your OpenAI API key>",
    },  # OpenAI API endpoint for gpt-4
    {
        "model": "gpt-35-turbo-0631",  # 0631 or newer is needed to use functions
        "base_url": "<your Azure OpenAI API base>", 
        "api_type": "azure", 
        "api_version": "2023-08-01-preview", # 2023-07-01-preview or newer is needed to use functions
        "api_key": "<your Azure OpenAI API key>"
    }
]
```

You can set the value of config_list in any way you prefer. Please refer to this [notebook](https://github.com/microsoft/autogen/blob/main/website/docs/llm_endpoint_configuration.ipynb) for full code examples of the different methods.

## Construct Agents

In [46]:
llm_config = {
    "timeout": 60,
    "cache_seed": 42,
    "config_list": config_list,
    "temperature": 0,
}
import json
terminology = {}
learning_objectives = {}
terms_to_learn = {}


with open('./data/termstolearn.json', 'r') as file:
    terminology = json.load(file)
with open('./data/learningobjectives.json', 'r') as file:
    learning_objectives = json.load(file)
with open('./data/termstolearn.json', 'r') as file:
    terms_to_learn = json.load(file)

# autogen.ChatCompletion.start_logging()


def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()


boss = autogen.UserProxyAgent(
    name="Boss",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    system_message="""You receive a question from a user. You are responsible for initiating the conversation and providing the user with the best possible answer. 
    You will receive recommendation from other agents. Use this to answer the user.
    template answer:
    -----------------
sentence given: *The sentence given*
my task: *Your evaluation or task*""",
    code_execution_config=False,  # we don't want to execute code in this case.
)

Initiate_conversation = autogen.AssistantAgent(
    name="Initiate_conversation",
    is_termination_msg=termination_msg,
    system_message="""You are a friend that that creates a good answer to the question. You also ask questions back to have an active conversation.
template answer:
-----------------
sentence given: *The sentence given or revised sentence*
my task: *Your evaluation or task*
""",
    llm_config=llm_config,
)

Learning_Objectives_Advisor = autogen.AssistantAgent(
    name="Learning_Objectives_Advisor",
    is_termination_msg=termination_msg,
    system_message="""You are an learning-objectives advisor that consideres the learning objectives. You tell what you would change in the sentence to achieve the learning objectives. Also tell why. If the sentence is too difficult or too easy, suggest a different answer to the question. 
    The learning objectives are: """ + json.dumps(learning_objectives)  + """
    edit the sentence given to what you would say.

template answer:
-----------------
sentence given: *The sentence given or revised sentence*
my task: *Your evaluation or task*""",
    llm_config=llm_config,
)

Terminology_Advisor = autogen.AssistantAgent(
    name="Terminology_Advisor",
    is_termination_msg=termination_msg,
    system_message="""You are a terminology advisor that keeps track of the terminology used in the conversation, and the terminology known by the user. If there is too much unknown terminology, you suggest what should be different. 
    It's best to not use too many unknown words. Let's say 40 percent of the sentence should be known terminology by the user. edit the sentence given to what you would say.
    The user knows the follwing terminology: """ + json.dumps(terminology)  + """
template answer:
-----------------
sentence given: *The sentence given or revised sentence*
my task: *Your evaluation or task*
""",
    llm_config=llm_config,
)

Critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=termination_msg,
    system_message="""You check if the answer is a good answer to the text given by the user.
    edit the sentence given to what you would say.
template answer:
-----------------
sentence given: *The sentence given or revised sentence*
my task: *Your evaluation or task*
""",
    llm_config=llm_config,
)


PROBLEM = "Hello! My name is Maria!"


def _reset_agents():
    boss.reset()
    Initiate_conversation.reset()
    Learning_Objectives_Advisor.reset()
    Terminology_Advisor.reset()
    Critic.reset()



In [40]:
_reset_agents()
groupchat = autogen.GroupChat(
    agents=[boss, Initiate_conversation, Learning_Objectives_Advisor, Terminology_Advisor, Critic],
    messages=[],
    max_round=12,
    speaker_selection_method="round_robin",
    allow_repeat_speaker=False,
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

# Start chatting with the boss as this is the user proxy agent.
boss.initiate_chat(
    manager,
    message=PROBLEM)

Boss (to chat_manager):

Hello! My name is Maria!

--------------------------------------------------------------------------------
Initiate_conversation (to chat_manager):

Hi Maria! Nice to meet you. What brings you here today?

--------------------------------------------------------------------------------
Learning_Objectives_Advisor (to chat_manager):

sentence given: "Hi Maria! Nice to meet you. What brings you here today?"
my task: The sentence is appropriate as it is. It aligns with the learning objectives of basic communication, interaction skills, and speaking and pronunciation. It initiates a simple conversation on a familiar topic (introductions) and uses familiar everyday expressions. The learner is also encouraged to respond, promoting interaction skills. The sentence is clear and easy to understand, aiding in listening comprehension. No changes are necessary.

--------------------------------------------------------------------------------
Terminology_Advisor (to chat_ma

### RetrieveUserProxyAgent get the correct code
Since RetrieveUserProxyAgent can perform retrieval-augmented generation based on the given documentation file, ChatGPT can generate the correct code for us!

In [47]:
PROBLEM = "I am here just working by myself. I went to a football game and the enemy team won. I am so sad. I am going to eat some ice cream to feel better. After the ice cream I will probably go watch some television and game a little bit."

_reset_agents()
groupchat = autogen.GroupChat(
    agents=[boss, Initiate_conversation, Learning_Objectives_Advisor, Terminology_Advisor, Critic],
    messages=[],
    max_round=12,
    speaker_selection_method="round_robin",
    allow_repeat_speaker=False,
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

# Start chatting with the boss as this is the user proxy agent.
boss.initiate_chat(
    manager,
    message=PROBLEM)

Boss (to chat_manager):

I am here just working by myself. I went to a football game and the enemy team won. I am so sad. I am going to eat some ice cream to feel better. After the ice cream I will probably go watch some television and game a little bit.

--------------------------------------------------------------------------------
Initiate_conversation (to chat_manager):

The sentence given or revised sentence: I spent the day working alone. I attended a football match but unfortunately, the opposing team won, which left me feeling quite down. To cheer myself up, I plan on indulging in some ice cream. After that, I'll likely spend the rest of my evening watching television and playing some video games.

My task: To provide a sympathetic response and engage in further conversation.

I'm sorry to hear that your team didn't win. It's always disappointing when that happens. But hey, ice cream sounds like a great way to lift your spirits! What flavor are you going to have? And what vide